In [1]:
from utils.data_utils import split_dataset, TimeSeriesDataset
from utils.evaluation_utils import plot_multistep_forecast
from utils.training_utils import train
import pandas as pd 
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [2]:
ETTm2 = pd.read_csv("/workspaces/time_series_experiment/ETT-small/ETTm2.csv")
training_df, test_df = split_dataset(ETTm2, remain_same = False)

In [4]:
p_lag = 96
future_steps = 48
batch_size = 8
epochs = 2
learning_rate=1.e-4
decomp_kernel_size = 24
number_of_forecasts = 100
target_column = ['OT']
feature_columns = [i for i in training_df.columns]
modelling_task = 'univariate'
n_continous_features=7
n_categorial_features=5
dataset_name = 'ETTm2LongTraining'
moe = True
number_of_experts = 2

In [6]:
net = train(
            epochs = epochs, 
            n_continous_features=n_continous_features, 
            n_categorial_features=n_categorial_features,
            p_lag=  p_lag, 
            future_steps = future_steps, 
            training_df = training_df, 
            validation_df = test_df, 
            feature_columns = feature_columns,
            target_column = target_column, 
            learning_rate=learning_rate ,
            decomp_kernel_size= decomp_kernel_size, 
            batch_size=batch_size, 
            model = 'rlinear', 
            modelling_task = modelling_task, 
            dataset_name = dataset_name, 
            moe = moe, 
            num_of_experts = number_of_experts
            )
test_data = DataLoader(TimeSeriesDataset(test_df, future_steps= future_steps, target_column = target_column,feature_columns=feature_columns,p_lag=p_lag), batch_size=batch_size,drop_last=True)
plot_multistep_forecast(test_data=test_data, dataset_name = dataset_name, neural_net=net, future_steps=future_steps, number_of_forecasts=number_of_forecasts)

Univatiate modelling
inflation factor = 1
Dlinear activated
Points to be estimated
With a shallow network
Univatiate modelling
inflation factor = 1
Dlinear activated
Points to be estimated
With a shallow network
Started training expert 1/2
Current learning rate is : 0.0001
---------------------------
Current (running) training loss at iteration 5000 : 292.0562657958984
Current (running) training loss at iteration 10000 : 244.352361529541
Current (running) training loss at iteration 15000 : 196.8894058186849
Current (running) training loss at iteration 20000 : 165.17679205932617
Current (running) training loss at iteration 25000 : 147.39051842041016
Current (running) training loss at iteration 30000 : 146.43503849995932
Current (running) training loss at iteration 35000 : 154.15224364100865
Current (running) training loss at iteration 40000 : 169.0848502799988
Current (running) training loss at iteration 45000 : 170.2218905456543


KeyboardInterrupt: 